# Часть 1. Проверка гипотезы в Python и составление аналитической записки



- Автор: Гуреев Конcтантин Викторович

## Цели и задачи проекта

<font color='#777778'>Наша цель провести А/В тестирование чтобы ответить на вопрос - пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении, чем пользователи из Москвы?
 
    Наши задачи:
- Проверить наличие дубликатов в идентификаторах пользователей
- Сравнить размеры групп, их статистики и распределение.
- Написать вывод на основе результатов теста.</font>

## Содержимое проекта

<font color='#777778'>Проект содержит следующие шаги:
- Проверка наличие дубликатов в идентификаторах пользователей
- Сравнение размеров групп и их статистики.
- Проверка гипотезы.
- Подготовка аналитической записки.</font>

---

## 1. Загрузка данных и знакомство с ними

In [1]:
# загрузим необходимые библиотеки
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import norm

In [2]:
# Ознакомимся с данными
user_df.head()

,Unnamed: 0,city,puid,hours
0,0,Москва,9668,26.167776
1,1,Москва,16598,82.111217
2,2,Москва,80401,4.656906
3,3,Москва,140205,1.840556
4,4,Москва,248755,151.326434


In [3]:
# Проверяем есть ли дубликаты
# Находим все строки, где puid повторяется
duplicates = user_df[user_df.duplicated(subset=['puid'], keep=False)]
#  Считаем количество уникальных пользователей
unique_users = user_df['puid'].nunique()
display(f"Всего уникальных пользователей: {unique_users}")

# Считаем общее количество записей
total_records = len(user_df)
display(f"Всего записей в таблице: {total_records}")

# Считаем количество записей с дубликатами
duplicate_count = len(duplicates)
display(f"Записей с дубликатами: {duplicate_count}")

# Оставляем только первую встречу puid
df_clean = user_df.drop_duplicates(subset=['puid'], keep='first')
# число строк после удаления дубликатов
display(f'число строк после удаления дубликатов {len(df_clean)}')

'Всего уникальных пользователей: 8540'

'Всего записей в таблице: 8784'

'Записей с дубликатами: 488'

'число строк после удаления дубликатов 8540'

In [4]:
# Подсчитаем количество пользователей по городам
city_counts = user_df['city'].value_counts()
display("Количество пользователей по городам:")
display(city_counts)

'Количество пользователей по городам:'

Москва             6234
Санкт-Петербург    2550
Name: city, dtype: int64

In [5]:
# Статистика по городам
display("Статистика по городам:")
city_stats = user_df.groupby('city').agg({
    'puid': 'nunique',  # Уникальные пользователи
    'hours': ['mean', 'sum', 'median', 'std']  # Статистика по часам
})
display(city_stats)

'Статистика по городам:'

puid      hours                                   
                nunique       mean           sum    median        std
city                                                                 
Москва             6234  10.881092  67832.727924  0.924498  36.851683
Санкт-Петербург    2550  11.592691  29561.361481  0.984781  39.704993

In [6]:
# Процентное соотношение пользователей по городам
city_percentage = city_counts / city_counts.sum() * 100
display("Процентное соотношение пользователей по городам:")
display(city_percentage)

'Процентное соотношение пользователей по городам:'

Москва             70.969945
Санкт-Петербург    29.030055
Name: city, dtype: float64

В связи с разным размером групп будем пользоваться тестом Уэлча.

In [7]:
user_df.duplicated().sum()

0

## 2. Проверка гипотезы в Python

Гипотеза звучит так: пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении, чем пользователи из Москвы. Попробуйте статистически это доказать, используя одностороннюю проверку гипотезы с двумя выборками:

- Нулевая гипотеза H₀: Средняя активность пользователей в часах в двух группах (Москва и Санкт-Петербург) не различается.

- Альтернативная гипотеза H₁: Средняя активность пользователей в Санкт-Петербурге больше, и это различие статистически значимо.

In [8]:
# Разделяем данные по городам
spb_data = user_df[user_df['city'] == 'Санкт-Петербург']['hours']
msk_data = user_df[user_df['city'] == 'Москва']['hours']

# Базовая статистика
print("Статистика по Санкт-Петербургу:")
print(f"Размер выборки: {len(spb_data)}")
print(f"Среднее время: {spb_data.mean():.2f} часов")
print(f"Стандартное отклонение: {spb_data.std():.2f} часов\n")

print("Статистика по Москве:")
print(f"Размер выборки: {len(msk_data)}")
print(f"Среднее время: {msk_data.mean():.2f} часов")
print(f"Стандартное отклонение: {msk_data.std():.2f} часов\n")

# Выполняем t-тест
# Поскольку выборки разного размера, используем Welch-тест
t_stat, p_value = stats.ttest_ind(
    spb_data, 
    msk_data, 
    equal_var=False,  # Welch-тест
    alternative='greater'  # Односторонний тест
)

print("Результаты t-теста:")
print(f"t-статистика: {t_stat:.4f}")
print(f"p-значение: {p_value:.4f}")

# Делаем вывод
alpha = 0.05  # Уровень значимости
if p_value < alpha:
    print("Гипотеза H0 отвергается. Есть статистически значимые доказательства того, что среднее время активности пользователей в Санкт-Петербурге больше, чем в Москве.")
else:
    print("Нет достаточных доказательств для отвержения H0.")

Статистика по Санкт-Петербургу:
Размер выборки: 2550
Среднее время: 11.59 часов
Стандартное отклонение: 39.70 часов

Статистика по Москве:
Размер выборки: 6234
Среднее время: 10.88 часов
Стандартное отклонение: 36.85 часов

Результаты t-теста:
t-статистика: 0.7782
p-значение: 0.2182
Нет достаточных доказательств для отвержения H0.


## 3. Аналитическая записка
По результатам анализа данных подготовьте аналитическую записку, в которой опишете:

- Выбранный тип t-теста и уровень статистической значимости.

- Результат теста, или p-value.

- Вывод на основе полученного p-value, то есть интерпретацию результатов.

- Одну или две возможные причины, объясняющие полученные результаты.




1. Тип статистического теста: односторонний t-тест Уэлча (Welch’s t-test) для независимых выборок с неравными дисперсиями.

Уровень статистической значимости: α = 0.05

2. Результаты статистического анализа

Гипотезы исследования:
- H0: Среднее время активности пользователей в Санкт-Петербурге не превышает среднее время в Москве (μСПб ≤ μМосква)
- H1: Среднее время активности пользователей в Санкт-Петербурге статистически значимо превышает время в Москве (μСПб > μМосква)

Статистические показатели:
- t-статистика: 0.7782
- p-значение: 0.2182

3. Интерпретация результатов

На основании полученного p-значения (0.2182 > 0.05) нет достаточных статистических доказательств для отвержения нулевой гипотезы. Это означает, что:
- Различия в среднем времени активности между городами статистически незначимы
- Нельзя утверждать, что пользователи из Санкт-Петербурга проводят больше времени в приложении, чем пользователи из Москвы

4. Возможные объяснения результатов

Размер выборки и дисперсия:

Большая дисперсия в обеих выборках (39.70 и 36.85 часов) может маскировать реальные различия

Малый размер и перекос в размерах групп может привести к искаженнуму результату статистического теста.

----

# Часть 2. Анализ результатов A/B-тестирования

Теперь вам нужно проанализировать другие данные. Представьте, что к вам обратились представители интернет-магазина BitMotion Kit, в котором продаются геймифицированные товары для тех, кто ведёт здоровый образ жизни. У него есть своя целевая аудитория, даже появились хиты продаж: эспандер со счётчиком и напоминанием, так и подстольный велотренажёр с Bluetooth.

В будущем компания хочет расширить ассортимент товаров. Но перед этим нужно решить одну проблему. Интерфейс онлайн-магазина слишком сложен для пользователей — об этом говорят отзывы.

Чтобы привлечь новых клиентов и увеличить число продаж, владельцы магазина разработали новую версию сайта и протестировали его на части пользователей. По задумке, это решение доказуемо повысит количество пользователей, которые совершат покупку.

Ваша задача — провести оценку результатов A/B-теста. В вашем распоряжении:

* данные о действиях пользователей и распределении их на группы,

* техническое задание.

Оцените корректность проведения теста и проанализируйте его результаты.

## 1. Опишите цели исследования.



- Провести оценку результатов A/B-теста.
- Оценить корректность проведения теста и проанализировать его результаты.

## 2. Загрузите данные, оцените их целостность.


## 3. По таблице `ab_test_participants` оцените корректность проведения теста:

   3\.1 Выделите пользователей, участвующих в тесте, и проверьте:

   - соответствие требованиям технического задания,

   - равномерность распределения пользователей по группам теста,

   - отсутствие пересечений с конкурирующим тестом (нет пользователей, участвующих одновременно в двух тестовых группах).

In [10]:
# Фильтруем только нужный тест
test_name = 'interface_eu_test'
test_df = participants[participants['ab_test'] == test_name]

In [11]:
# 1. Проверка соответствия ТЗ
print("Проверка соответствия техническому заданию:")
# Проверка наличия только нужных групп
valid_groups = test_df['group'].unique()
required_groups = ['A', 'B']
if set(valid_groups) == set(required_groups):
    print("✓ Группы соответствуют ТЗ (A и B присутствуют)")
else:
    print("❌ Ошибка: обнаружены недопустимые группы")

Проверка соответствия техническому заданию:
✓ Группы соответствуют ТЗ (A и B присутствуют)


In [12]:
# 2. Проверка равномерности распределения
group_distribution = test_df.groupby('group').agg({
    'user_id': 'nunique'
}).reset_index()
print("\nРаспределение пользователей по группам:")
print(group_distribution)
# Расчет разницы в количестве пользователей
group_counts = group_distribution.set_index('group')['user_id']
difference = abs(group_counts['A'] - group_counts['B'])
total_users = group_counts.sum()
print(f"\nРазница в количестве пользователей между группами: {difference}")
print(f"Общее количество участников теста: {total_users}")


Распределение пользователей по группам:
  group  user_id
0     A     5383
1     B     5467

Разница в количестве пользователей между группами: 84
Общее количество участников теста: 10850


In [13]:
# 3. Проверка на пересечения с другими тестами
# Создаем набор всех user_id в текущем тесте
test_users = set(test_df['user_id'])
# Разделяем пользователей по группам A и B
group_a_users = set(test_df[test_df['group'] == 'A']['user_id'])
group_b_users = set(test_df[test_df['group'] == 'B']['user_id'])

# Проверяем пересечения между группами A и B
ab_overlap = group_a_users.intersection(group_b_users)

if ab_overlap:
    print("\n❌ Обнаружены пересечения между группами A и B!")
    print(f"Количество пересекающихся пользователей: {len(ab_overlap)}")
    print("Список пересекающихся пользователей:")
    print(test_df[test_df['user_id'].isin(ab_overlap)]['user_id'].unique())
else:
    print("\n✓ Пересечений между группами A и B нет")
# Получаем всех пользователей, которые находятся в тестовых группах других тестов
other_tests = test_df[(test_df['ab_test'] == 'recommender_system_test') & (test_df['group'] == 'B')]

# Находим пересечения именно с тестовыми группами других тестов
overlapping_test_users = set(other_tests['user_id'])

# Проверяем пересечения
if test_users.intersection(overlapping_test_users):
    print("\n❌ Обнаружены пересечения с тестовыми группами других тестов!")
    print(f"Количество пересекающихся пользователей: {len(test_users.intersection(overlapping_test_users))}")
    
    # Дополнительно выведем список пересекающихся пользователей
    print("\nСписок пересекающихся пользователей:")
    print(test_df[test_df['user_id'].isin(overlapping_test_users)]['user_id'].unique())
else:
    print("\n✓ Пересечений с тестовыми группами других тестов нет")


✓ Пересечений между группами A и B нет

✓ Пересечений с тестовыми группами других тестов нет


In [14]:
# 4. Проверка на дубликаты
duplicates = test_df.duplicated(subset='user_id', keep=False)
if duplicates.any():
    print("\n❌ Обнаружены дублирующиеся user_id")
    print(f"Количество дубликатов: {duplicates.sum()}")
else:
    print("\n✓ Дубликатов user_id не обнаружено")


✓ Дубликатов user_id не обнаружено


In [15]:
# Итоговый вывод
print("\nРезультаты проверки соответствия ТЗ:")
if (set(valid_groups) == set(required_groups) and 
    not duplicates.any() and 
    not test_users.intersection(overlapping_test_users)):
    print("✓ Тест полностью соответствует техническому заданию")
else:
    print("❌ Тест не соответствует техническому заданию")


Результаты проверки соответствия ТЗ:
✓ Тест полностью соответствует техническому заданию


3\.2 Проанализируйте данные о пользовательской активности по таблице `ab_test_events`:

- оставьте только события, связанные с участвующими в изучаемом тесте пользователями;

In [16]:
# Фильтруем только нужных участников для нашего теста
test_users = participants[
    (participants['ab_test'] == test_name)
]['user_id'].unique()

In [17]:
# Фильтруем таблицу с событиями и оставляем только информацию с пользователями учавствующими с тесте
filtered_events = events[events['user_id'].isin(test_users)]


In [18]:
# Изучим результат
filtered_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79715 entries, 64672 to 780371
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     79715 non-null  object        
 1   event_dt    79715 non-null  datetime64[ns]
 2   event_name  79715 non-null  object        
 3   details     21075 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 3.0+ MB


In [19]:
filtered_events.head()

,user_id,event_dt,event_name,details
64672,5F506CEBEDC05D30,2020-12-06 14:10:01,registration,0.0
64946,51278A006E918D97,2020-12-06 14:37:25,registration,-3.8
66585,A0C1E8EFAD874D8B,2020-12-06 17:20:22,registration,-3.32
67873,275A8D6254ACF530,2020-12-06 19:36:54,registration,-0.48
67930,0B704EB2DC7FCA4B,2020-12-06 19:42:20,registration,0.0


- определите горизонт анализа: рассчитайте время (лайфтайм) совершения события пользователем после регистрации и оставьте только те события, которые были выполнены в течение первых семи дней с момента регистрации;

In [20]:
# Выделяем события регистрации
registration_events = filtered_events[filtered_events['event_name'] == 'registration']

# Создаем словарь времени регистрации
registration_times = registration_events.groupby('user_id')['event_dt'].min().reset_index()
registration_times.rename(columns={'event_dt': 'registration_dt'}, inplace=True)

# Выделяем события покупки
purchase_events = filtered_events[filtered_events['event_name'] == 'purchase']


In [21]:
# Объединяем с временем регистрации
purchase_with_registration = pd.merge(
 purchase_events,
 registration_times,
 on='user_id'
)

# Рассчитываем время с момента регистрации
purchase_with_registration['days_since_registration'] = (
 purchase_with_registration['event_dt'] - 
 purchase_with_registration['registration_dt']
).dt.days

In [22]:
# Оставляем только покупки в течение 7 дней после регистрации
valid_purchases = purchase_with_registration[purchase_with_registration['days_since_registration'] < 7]

# Базовая статистика по покупкам
print("Статистика покупок за 7 дней:")
print(valid_purchases.info())

# Распределение покупок по дням
purchase_day_distribution = valid_purchases.groupby('days_since_registration').size()
print("\nРаспределение покупок по дням с момента регистрации:")
print(purchase_day_distribution)

# Конверсия по дням
total_registered = len(registration_times)
purchased_users = valid_purchases['user_id'].nunique()
conversion_rate = purchased_users / total_registered * 100

print(f"\nКонверсия в покупку за 7 дней: {conversion_rate:.2f}%")

Статистика покупок за 7 дней:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6623 entries, 0 to 10171
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   user_id                  6623 non-null   object        
 1   event_dt                 6623 non-null   datetime64[ns]
 2   event_name               6623 non-null   object        
 3   details                  6623 non-null   object        
 4   registration_dt          6623 non-null   datetime64[ns]
 5   days_since_registration  6623 non-null   int64         
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 362.2+ KB
None

Распределение покупок по дням с момента регистрации:
days_since_registration
0     870
1    1033
2    1120
3    1046
4     963
5     895
6     696
dtype: int64

Конверсия в покупку за 7 дней: 28.39%


Оцените достаточность выборки для получения статистически значимых результатов A/B-теста. Заданные параметры:

- базовый показатель конверсии — 30%,

- мощность теста — 80%,

- достоверность теста — 95%.

In [23]:
# Задаем параметры теста
base_conversion = 0.3  # базовый показатель конверсии
effect_size = 0.03     # минимальный детектируемый эффект (3 процентных пункта)
power = 0.8            # мощность теста (80%)
alpha = 0.05           # уровень значимости (95% достоверность)

# Рассчитываем необходимый размер выборки
# Используем формулу для пропорций
effect = proportion_effectsize(prop1=base_conversion, 
                                  prop2=base_conversion + effect_size)

analysis = NormalIndPower()
result = analysis.solve_power(effect_size=effect, 
                             power=power, 
                             alpha=alpha, 
                             ratio=1,  # равные группы
                             alternative='two-sided')

# Рассчитываем общее количество наблюдений
n_per_group = int(np.ceil(result))  # округляем в большую сторону
total_sample_size = n_per_group * 2  # умножаем на 2, так как две группы

print(f"Параметры теста:")
print(f"  Базовый показатель конверсии: {base_conversion*100}%")
print(f"  Минимальный детектируемый эффект: {effect_size*100}%")
print(f"  Мощность теста: {power*100}%")
print(f"  Достоверность: {(1-alpha)*100}%")

Параметры теста:
  Базовый показатель конверсии: 30.0%
  Минимальный детектируемый эффект: 3.0%
  Мощность теста: 80.0%
  Достоверность: 95.0%


- рассчитайте для каждой группы количество посетителей, сделавших покупку, и общее количество посетителей.

In [24]:
print(f"Необходимый размер выборки для каждой группы: {n_per_group}")
print(f"Общий необходимый размер выборки: {total_sample_size}")

Необходимый размер выборки для каждой группы: 3762
Общий необходимый размер выборки: 7524


- сделайте предварительный общий вывод об изменении пользовательской активности в тестовой группе по сравнению с контрольной.

In [25]:
# Добавляем информацию о группах к событиям
valid_purchases = pd.merge(
    valid_purchases,
    participants[['user_id', 'group']],
    on='user_id')
# Получаем всех зарегистрированных пользователей по группам
registered_u = registration_times.merge(
    participants[['user_id', 'group']],
    on='user_id'
)

# зарегистрированных в каждой группе
registered_a = registered_u[registered_u['group'] == 'A']
registered_b = registered_u[registered_u['group'] == 'B']

In [26]:
# Разделяем на группы совершивших покупку
group_a = valid_purchases[valid_purchases['group'] == 'A']
group_b = valid_purchases[valid_purchases['group'] == 'B']

In [27]:
# Метрики для групп
def calculate_metrics(group, registered):
    metrics = {
        'unique_users': registered['user_id'].nunique(),
        'purchases': group['user_id'].nunique(),
        'conversion': group['user_id'].nunique() / registered['user_id'].nunique(),
        'avg_days_to_purchase': group['days_since_registration'].mean(),
        'purchases_per_user': len(group) / group['user_id'].nunique()
    }
    return pd.Series(metrics)

metrics_a = calculate_metrics(group_a, registered_a)
metrics_b = calculate_metrics(group_b, registered_b)

In [28]:
# Рассчитываем разницу в процентных пунктах для конверсии
conversion_diff_pp = (metrics_b['conversion'] - metrics_a['conversion']) * 100

# Сравнение метрик
comparison = pd.DataFrame({
    'Group A': metrics_a,
    'Group B': metrics_b,
    'Difference': metrics_b - metrics_a,
    'Relative Change (%)': (metrics_b - metrics_a) / metrics_a * 100,
    'Difference (pp)': (metrics_b - metrics_a) * 100  # Добавляем колонку с разницей в п.п.
})

print("Сравнение метрик по группам:")
print(comparison)

# Отдельно выводим разницу в процентных пунктах для конверсии
print(f"\nРазница в конверсии в процентных пунктах: {conversion_diff_pp:.2f} п.п.")

Сравнение метрик по группам:
                          Group A      Group B  Difference  \
unique_users          5723.000000  5573.000000 -150.000000   
purchases             1579.000000  1626.000000   47.000000   
conversion               0.275904     0.291764    0.015860   
avg_days_to_purchase     2.770407     2.919748    0.149342   
purchases_per_user       2.071564     2.345018    0.273454   

                      Relative Change (%)  Difference (pp)  
unique_users                    -2.621003    -15000.000000  
purchases                        2.976567      4700.000000  
conversion                       5.748232         1.585962  
avg_days_to_purchase             5.390603        14.934163  
purchases_per_user              13.200371        27.345417  

Разница в конверсии в процентных пунктах: 1.59 п.п.


## 4. Проведите оценку результатов A/B-тестирования:

- Проверьте изменение конверсии подходящим статистическим тестом, учитывая все этапы проверки гипотез.

In [29]:
# Получаем всех зарегистрированных пользователей по группам
registered_users = registration_times.merge(
    participants[['user_id', 'group']],
    on='user_id'
)

# Считаем количество уникальных зарегистрированных в каждой группе
total_registered_a = registered_users[registered_users['group'] == 'A']['user_id'].nunique()
total_registered_b = registered_users[registered_users['group'] == 'B']['user_id'].nunique()

# Считаем количество совершивших покупку
converted_a = group_a['user_id'].nunique()
converted_b = group_b['user_id'].nunique()


# Рассчитываем конверсию
conversion_a = converted_a / total_registered_a
conversion_b = converted_b / total_registered_b

print("Проверка данных:")
print(f"Зарегистрировано в группе A: {total_registered_a}")
print(f"Покупок в группе A: {converted_a}")
print(f"Зарегистрировано в группе B: {total_registered_b}")
print(f"Покупок в группе B: {converted_b}")



Проверка данных:
Зарегистрировано в группе A: 5723
Покупок в группе A: 1579
Зарегистрировано в группе B: 5573
Покупок в группе B: 1626


Формулируем гипотезы:
- H0: конверсия в группе B не выше, чем в группе A на 3 п.п.
- H1: конверсия в группе B выше, чем в группе A минимум на 3 п.п.


In [30]:
# Собираем данные для теста
count = np.array([converted_a, converted_b])
nobs = np.array([total_registered_a, total_registered_b])

try:
    # Проводим z-тест
    z_stat, p_value = proportions_ztest(
        count=count,
        nobs=nobs,
        value=0.03,  # ожидаемое минимальное увеличение 3 п.п.
        alternative='larger'
    )
    
    print("\nРезультаты теста:")
    print(f"Z-статистика: {z_stat:.4f}")
    print(f"P-значение: {p_value:}")
    
    # Делаем выводы
    if p_value < 0.05 and (conversion_b - conversion_a) >= 0.03:
        print("\nГипотеза подтверждена!")
        print(f"Конверсия в группе B выше на {(conversion_b - conversion_a)*100:.2f}%")
    elif p_value < 0.05:
        print("\nОбнаружены статистически значимые различия")
        print(f"Но эффект меньше ожидаемого: {(conversion_b - conversion_a)*100:.2f}%")
    else:
        print("\nСтатистически значимых различий не обнаружено")
        
except Exception as e:
    print(f"Ошибка при проведении теста: {e}")

# Итоговая статистика
print("\nИтоговая конверсия:")
print(f"Группа A: {conversion_a:.2%}")
print(f"Группа B: {conversion_b:.2%}")
print(f"Разница: {conversion_b - conversion_a:.2%}")


Результаты теста:
Z-статистика: -5.4055
P-значение: 0.9999999676819358

Статистически значимых различий не обнаружено

Итоговая конверсия:
Группа A: 27.59%
Группа B: 29.18%
Разница: 1.59%


- Опишите выводы по проведённой оценке результатов A/B-тестирования. Что можно сказать про результаты A/B-тестирования? Был ли достигнут ожидаемый эффект в изменении конверсии?

- Z-статистика =  -5.4055 (отрицательное значение указывает на то, что группа B группа B не показала существенно лучших результатов)

Вывод: статистически значимых различий между группами не обнаружено

Конверсия в группе A (контрольная): 27.59%

Конверсия в группе B (тестовая): 29.18%

Абсолютная разница: +1.59% в пользу тестовой группы.

Наблюдается положительная динамика конверсии в тестовой группе

Хотя конверсия в группе B выше на 1.59%, это увеличение:

Не достигает минимально ожидаемого уровня в 3 п.п.

Не является статистически значимым

Т.к. разница в конверсии ниже ожидаемой рекомендуется рассмотреть возможность корректировки тестируемого решения.